In [23]:
import json
import logging
from dateutil import parser
import pandas as pd
import logging
import datetime
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import re
import ntpath
import dateutil


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import DLP

In [25]:
## GLOBAL VARS
SEARCH_DATA_SYNID = 'syn11377377'
LOCATION_DATA_SYNID = 'syn11377380'
WEBVISTIS_SYNID = 'syn11977096'
ENROLLMENT_SYNTABLE = 'syn11384434'

In [26]:
def process_userSearchQueries_in_htmlFormat(html_file_content):
    '''
    html_file_content - is the content (string) of the HTML file
    '''

    textSearches = []
    webVisits = []
    errorBlock = []
    #this should be a unique search block in HTML file
    blocks = re.findall('<div class="outer-cell.*?</div></div></div>',html_file_content)
    #process each block    
    for b in blocks:
        textSearch,webVisit = processSearchBlock(b)
        if textSearch is None and webVisit is None:
            errorBlock.append(b)
        if textSearch: textSearches.append(textSearch) 
        if webVisit: webVisits.append(webVisit)
    webVisits_df = pd.DataFrame.from_records(webVisits,columns=('weblink', 'localTimestamp', 'location'))
    textSearches_df = pd.DataFrame.from_records(textSearches,columns=('query', 'localTimestamp', 'location'))
    return(webVisits_df, textSearches_df)


def processSearchBlock(block):
    #website_visits_match = re.match('^.+Visited.+?">(.*)/</a><br>(.*?)</div>.+$', block)
    webVisit       = re.match('^.*Visited.*">(.*)</a><br>(.*?)</div.*$', block)
    textSearch     = re.match('^.+?Searched for.+?">(.+?)</a><br>(.*?)</div>.+$', block)
    location       = re.match('^.*Locations.*">(.*)</a><br></div></div></div>$', block)
    
    if location:
        location = location.groups()
    else:
        location = ('NA',)
    
    if textSearch:
        textSearch = textSearch.groups()
        textSearch = textSearch + location
    else:
        textSearch = None
    
    if webVisit:
        webVisit = webVisit.groups()
        webVisit = webVisit + location
    else:
        webVist = None
    return([textSearch,webVisit])


In [97]:

fileName = '/Users/apratap/Downloads/test_takeOut_data/9129_takeout-20180807T222047Z-001.zip'


'extIDNA_afsID9129_date2018-08-07_time22-20-47'

### Process WebSearches

In [100]:

def _tmp_process_manually_downloaded_takeoutFile(takeoutFile):
    
    ## metadata extraction
    re_match = re.match('^(.*?)_takeout-(.*?)-001.zip$', ntpath.basename(fileName)).groups()
    extID = 'NA'
    afsID = re_match[0]
    consentTime = dateutil.parser.parse(re_match[1])
    tmp_date = consentTime.strftime('%Y-%m-%d')
    tmp_time = consentTime.strftime('%H-%M-%S')
    prefix = 'extID%s_afsID%s_date%s_time%s' % (extID,afsID, tmp_date, tmp_time)

    ###read and process the zip file
    x = ZipFile(fileName)
    search_files = [f for f in x.namelist() if 'Search' in f]
    search_file = search_files[0]
    html_file_content = x.read(search_file).decode('utf-8')
    webVisits, searchQueries = process_userSearchQueries_in_htmlFormat(html_file_content)

    #until we have a plan to handle this better 
    searchQueries = searchQueries.drop(['location'], axis=1)
    webVisits = webVisits.drop(['location'], axis=1)

    ### Run DLP API on Search Queries
    searchQueriesList = searchQueries['query'].tolist()
    dlp_results = DLP.run(searchQueriesList) 

    #integrate with queries
    searchQueries = searchQueries.merge(dlp_results, how='left', left_on='query', right_on='searchQuery')
    searchQueries = searchQueries.drop(['searchQuery'], axis=1)
    searchQueries['query'][searchQueries.redacted == True ] = 'REDACTED'
    
    ###Upload data to synapse
    searchQueriesFile = '%s_searchQueries.feath' % user.prefix
    searchQueries.to_feather(searchQueriesFile)
    tmp = syn.store(synapseclient.File(searchQueriesFile, parentId = SEARCH_DATA_SYNID))
    os.unlink(searchQueriesFile)
    
    
    
    
    
    
    
    ### Process Locations
    location_files = [f for f in x.namelist() if 'Location History' in f]
    if len(location_files) > 0:
        locations = pd.read_json(x.open(location_files[0]).read())
        
        

In [101]:
_tmp_process_manually_downloaded_takeoutFile(fileName)